
# Using [Hugging Face Models](https://huggingface.co/models)

### [Meta (Llama) Models](https://huggingface.co/meta-llama)

In [1]:
%pip install -q torch==2.5.1
#%pip install -q torch==2.5.1+cu124
%pip install -q transformers==4.46.2
%pip install -q accelerate
%pip install -q bitsandbytes
%pip install -q huggingface_hub

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import transformers
import accelerate
import bitsandbytes
import huggingface_hub
from transformers import pipeline
from huggingface_hub import login

print("PyTorch version:", torch.__version__)
print("Transformers version:", transformers.__version__)
print("Accelerate version:", accelerate.__version__)
print("Bitsandbytes version:", bitsandbytes.__version__)
print("Hugging Face Hub version:", huggingface_hub.__version__)

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


PyTorch version: 2.5.1+cpu
Transformers version: 4.46.2
Accelerate version: 1.1.1
Bitsandbytes version: 0.44.1
Hugging Face Hub version: 0.26.2


In [3]:
login()

#### Instruction Model

In [4]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"

pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    #torch_dtype=torch.float32,
    device_map="auto",
)
messages = [
    {"role": "system", "content": "You are a helpful chatbot always ready to help!"},
    {"role": "user", "content": "How many countries are in Europe? Please ennumerate"},
]
outputs = pipe(
    messages,
    max_new_tokens=600,
)

print(outputs[0]["generated_text"][-1]['content'])

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

C:\Users\Renato Rocha Souza\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Renato Rocha Souza\.cache\huggingface\hub\models--meta-llama--Llama-3.2-1B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


There are 50 countries in Europe. Here's the list:

1. Albania
2. Andorra
3. Armenia
4. Austria
5. Azerbaijan
6. Belarus
7. Belgium
8. Bosnia and Herzegovina
9. Bulgaria
10. Croatia
11. Cyprus
12. Czech Republic
13. Denmark
14. Estonia
15. Finland
16. France
17. Georgia
18. Germany
19. Greece
20. Hungary
21. Iceland
22. Ireland
23. Italy
24. Kazakhstan (partially in Asia)
25. Kosovo
26. Latvia
27. Liechtenstein
28. Lithuania
29. Luxembourg
30. Malta
31. Moldova
32. Monaco
33. Montenegro
34. Netherlands
35. North Macedonia
36. Norway
37. Poland
38. Portugal
39. Romania
40. Russia (partially in Asia)
41. San Marino
42. Serbia
43. Slovakia
44. Slovenia
45. Spain
46. Sweden
47. Switzerland
48. Turkey (partially in Asia)
49. Ukraine
50. United Kingdom (includes England, Scotland, Wales, and Northern Ireland)

Note: The status of some countries, like Kosovo and Turkey, is disputed, but they are generally considered part of Europe.


In [5]:
context = '''
Artificial intelligence systems like ChatGPT could soon run out of what keeps making them smarter —
the tens of trillions of words people have written and shared online.

A new study released Thursday by research group Epoch AI projects that tech companies will exhaust
the supply of publicly available training data for AI language models by roughly the turn of the decade
-- sometime between 2026 and 2032.

Comparing it to a “literal gold rush” that depletes finite natural resources, Tamay Besiroglu, an
author of the study, said the AI field might face challenges in maintaining its current pace of progress
once it drains the reserves of human-generated writing.
'''


messages = [
    {"role": "system", "content": "You are a linguist expert, specialized in syntax"},
    {"role": "user", "content": "What are the verbs of the text below?"},
    {"role": "user", "content": f"<text>{context}</text>"},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)

print(outputs[0]["generated_text"][-1]['content'])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


The verbs in the given text are:

1. could
2. run
3. keep
4. making
5. released
6. projects
7. exhaust
8. depletes
9. comparing
10. depletes
11. facing
12. maintaining
13. drains
14. written
15. shared
16. compared
17. said
18. projects



##### Creating a function

In [6]:
def ask(prompt, max_new_tokens=256):
    messages = [
    {"role": "system", "content": "You are a helpful assistant that follows instructions closely"},
    {"role": "user", "content": f"<instructions>{prompt}</instructions>"},
    ]
    outputs = pipe(
        messages,
        max_new_tokens=max_new_tokens,
    )
    return outputs[0]["generated_text"][-1]['content']

In [7]:
print(ask("What is the size of planet earth?", 240))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


To answer your question, I'll follow the instructions closely.

According to various sources, including NASA and the United Nations, the size of Earth is approximately 12,742 kilometers (7,918 miles) in diameter.

Would you like to know more about Earth or its size in a different unit (e.g., kilometers, miles, or meters)?


In [ ]:
template = """Write a short summary of this article for a business expert:

{article}
"""

cameroon = """The economic fallout from the COVID-19 pandemic and the subsequent
global shocks provoked by the war in Ukraine have hit African countries hard,
denting economic growth and aggravating their sovereign debt positions. The
International Monetary Fund (IMF) forecasts that Cameroon, a Central African oil
producer, will record 4.3% economic growth this year after it slumped to 0.5% in
2020. The Fund has classified Cameroon as being at high risk of debt distress,
though in its most recent review of the country's loan programme it stated that,
with active fiscal reforms and management, the debt could be sustainable. "Our
debt service coverage from exports needs to be improved. That's the reason why
we are ranked in a high risk debt distress position," said Alamine Ousmane Mey,
Cameroon's minister of economy, planning and regional development. He was
speaking at an event organised by the Atlantic Council think tank on the
sidelines of the IMF and World Bank's Spring Meetings in Washington. "We're
working to be able to improve our exports through import substitution policies
to reduce imports, produce more and export more. This will give us better room
for debt service coverage," he said. Cameroon has also relaunched talks with the
U.S. to end its suspension from the Africa Growth and Opportunities Act (AGOA)
initiative, which grants qualifying African countries tariff-free access to the
U.S. market. Former President Donald Trump suspended Cameroon from the programme
in late 2019 over "persistent gross violations of internationally recognised
human rights" by Cameroonian security forces. Since 2017, factions of secessionist
militias have been battling government troops in the majority Francophone country's
two English-speaking regions. The conflict has killed thousands and displaced
nearly 800,000 people. "All the issues that have been raised, we're working on in
a very transparent open manner to be able to iron them out and solve the problems,"
Mey said, referring to the talks with U.S. officials to rejoin AGOA. Our Standards:
The Thomson Reuters Trust Principles.
"""

ask(template.format(article=cameroon), max_new_tokens=120)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
brazil = """Brazil's inflation likely stayed high in March on rising gasoline
bills, reigniting cost of living problems in the country's stagnant economy and
probably stoking more disagreement over policy, a Reuters poll showed. Consumer
prices cooled in the second half of 2022 in reaction to an aggressive tightening
campaign by the central bank. But inflation pressures reemerged after President
Luiz Inacio Lula da Silva took office at the start of this year. The monthly
report on consumer prices scheduled for Tuesday could again push back expectations
for policy easing towards the end of this year and reinforce Lula's view the
Bank's hawkish approach is proving ineffective. The benchmark consumer price index
IPCA increased 0.77% in March, according to the median estimate of 13 economists
polled March 29-April 3. This would stand very close to the 0.84% rate in February,
which had been the quickest in 10 months. "Food inflation should remain at low
levels for this time of the year... (but) the partial resumption of federal taxes
on gasoline and ethanol on March 1 will take a toll in the March IPCA," Morgan
Stanley analysts wrote in a report. The reinstatement of taxes on fuels already
tinged March mid-month inflation figures, which came in higher than forecast,
dampening any expectations Banco Central do Brasil may begin to cut its benchmark
rate, currently at 13.75%, in the short term. The 12-month reading for last month
is seen at 4.70%, under 5.60% in February and the lowest in more than two years
due to base effects. Brazil's IBGE statistics agency will publish the data on
Tuesday at 0900 local time (1200 GMT). A 0.77% inflation rate in March would
result in a cumulative 2.2% clip in the first quarter, well on course to surpass
this year's goal of 3.25% with a margin of 1.5 percentage points. Last week Lula
hinted the government may seek changes in the target to allow for rate cuts sooner,
an idea central bank governor Roberto Campos Neto - a member of a committee that
sets the goal - has rejected. The panel is comprised by the bank's chief and the
ministers of finance and planning, who think the fuel tax and other revenue-boosting
planned measures will help lower inflation by eventually closing the primary deficit.
Reporting and polling by Gabriel Burin; Editing by Jan Harvey Our Standards: T
he Thomson Reuters Trust Principles. Read Next United States category Fox News
settles with Venezuelan businessman in election defamation lawsuit , article with
image 9:19 AM UTC Macro Matters category Bank of Canada seen on hold even as
economy accelerates , article with gallery 9:59 AM UTC Markets category Canadian
miner Teck Resources pushes for restructuring, says Glencore bid 'flawed' ,
article with image 11:22 AM UTC Business category Mexico's automotive production
and exports rise in March , article with image 12:15 PM UTC
"""

ask(template.format(article=brazil, max_new_tokens=120))

In [ ]:
argentina = """Homeless in Argentina seek refuge in unofficial shelter in Buenos
Aires airport At the start of the long Easter weekend, the airport in the Argentine
capital is filled with homeless people sleeping. The Jorge Berry domestic airport
has become an unofficial homeless shelter in a reflection of the rising poverty
in Argentina, which also has one of the world’s highest inflation rates. More and
more Argentines are finding themselves in a difficult financial situation as the
country’s inflation clocked in at an annual rate of 102.5% in February. Although
the country has been used to double-digit inflation for years, this marked the
first time the annual rise in consumer prices reached triple digits since 1991.
The high inflation, which has been particularly pronounced in basic food items,
has hit the poor the hardest and pushed the poverty rate to almost 40 percent.
Nearly 18.6 million people are unable to cover their basic food needs, according
to official figures.
"""

ask(template.format(article=argentina, max_new_tokens=120))

In [ ]:
facc = """Nach geprüften Zahlen erzielte FACC einen Umsatz von 607,0 Mio. Euro
und damit rund 7 Mio. Euro mehr als Ende Februar avisiert (+22,0% yoy). Damit
erreicht das Unternehmen die Guidance und kann von den steigenden Produktionsraten
aller Flugzeug-OEMs profitieren. In Q4 verzeichnete das Unternehmen demnach einen
deutlichen Erlösanstieg von 34,5% yoy, der auch auf hohe Kundenzahl-ungen im
Zusammenhang mit Entwicklungsdienstleistungen im Schlussquartal zurückzu-führen
ist. Mit einem Top Line Wachstum von 40,3% yoy auf Ganzjahressicht trug das Segment
Aerostructures den Löwenteil zum Wachstum bei (235,1 Mio. Euro). Auch das im Umbau
befindliche Segment mit Innenausstattungen konnte um 21,3% yoy zulegen (274,6 Mio.
Euro). Das Geschäft mit Triebwerksbauteilen („Engines & Nacelles“) wies mit 6,1%
yoy eine rückläufige Top-Line auf (97,3 Mio. Euro). Ursächlich hierfür war der
auf Sicherheits-mängel zurückzuführende temporäre Auslieferungsstopp von Boeing
787-Maschinen. Alle Segmente mit positivem EBIT-Beitrag: Das EBIT von 5,5 Mio.
Euro liegt ebenfalls etwas oberhalb des vorab kommunizierten Wertes von 5,0 Mio.
Euro. Damit kehrt FACC nach den starken Beeinträchtigungen der Corona-Pandemie
auf operativer Ebene auf den Profitabilitätspfad zurück, wenngleich dies eine
Verfehlung des avisierten zweistelligen EBITs darstellt. Erfreulich ist die Rückkehr
des Segmentes Cabin Interiors zu einem positiven EBIT-Beitrag von 1,0 Mio. Euro
(Marge: 0,4%; Vj.: -2,3%). Mit einer Marge von 1,7% (Vj.: -17,0%) war das Segment
Aerostructures das profitabelste Geschäftsfeld. Die Profitabilität bei Engines &
Nacelles war durch den Sondereffekt im Zusammenhang mit dem 787-Lieferstopp
beeinträchtigt (0,5%; Vj.: 8,4%). Durch die hohen Meilensteinzahlungen für
Entwicklungstätigkeiten in Q4 fiel der Free Cashflow auf Konzernebene mit -0,07
Euro je Aktie deutlich besser aus als von uns erwartet (-0,74 Euro). Fortsetzung
des Erholungskurses in 2023 erwartet: Für 2023 avisiert FACC weiterhin von den
steigenden Produktionsraten der OEMs profitieren zu können und Umsatz und EBIT
zu steigern. Diese bewusst allgemein formulierte Guidance ist u.E. auf die hohe
Unsicherheit in der Lieferkette zurückzuführen, die sich u.a. in der bisher
geringen Lieferrate von Airbus manifestiert. Mit 66 Auslieferungen (Stand: Februar)
liegt Airbus deutlich hinter dem Vorjahreswert von 79 (-16,5%). Ergebnisseitig
hat FACC Maßnahmen ergriffen, um die signifikant gestiegenen Bezugspreise zu
kompensieren. So dürften diese durch Preiserhöhungen im Jahresverlauf an Kunden
teilweise weitergereicht werden. Außerdem erwarten wir in 2023 weitere Kosteneinsparungen
i.H.v. ca. 4,0 Mio. Euro durch die nach Kroatien verlagerte Produktion des Segmentes
Cabin Interiors (2022: ca. 1,5 Mio. Euro)."""

ask(template.format(article=facc), max_new_tokens=120)

In [ ]:
ups = """Any time you open up a mailbox on the UPS Retailer, you can get lots
more than simply a box having a key. You'll get bundle acceptance from all
shipping carriers, mail receipt notifications, and an actual street deal with
in 32904, not just a PO Box #. Implement these days. Specialising in both standard
and express freight supply, we're dedicated to making certain that the standard
of Just about every delivery is rarely compromised. They're able to function an
agent on behalf of your shipper by negotiating the pricing and the contract’s
conditions and terms with freight carriers. FreightPros has negotiated low-cost
freight quotations Using these and a number of other carriers across the country.
Our truckload group has secured associations with a lot of of those freight services
organizations also to help you our prospects ship their freight in both the LTL
and truckload shipping and delivery zones. Etihad Rail has announced the commencement
of its commercial freight services, which might be now entirely operational and
obtainable for transportation of all types of cargo across the UAE. You’ll need
to have to learn if you might want to request a elevate gate (the system on the
truck that raises and lowers freight to and from the ground towards the trailer)
for pickup and shipping. For those who or your location doesn’t Have a very delivery
dock, be sure you ask for a elevate gate to stay away from delays and additional
charges. But you don’t should go it by yourself. “It’s a misconception that ecommerce
providers require to obtain relatively deep expertise in international laws and
the whole process of transport internationally to achieve success with Global
sourcing,” states Cory Sanderson, Director of Consumer Solutions at Flexport, an
entire-provider freight forwarding and customs brokerage corporation. We heard
you, and we have improved our quoting working experience with a guided perspective
that will help you ship far more confidently, despite your standard of shipping
and delivery information
If your shipment requires a paper Bill of Lading, or if you prefer to implement
a Invoice of Lading rather of making a label on the net, you'll be able to download
a type. By publishing this form, you comply with receive marketing messages from
Shopify about its items and services. You can unsubscribe at any time by clicking
over the link at the bottom of our e-mails. Considered one of CargoMasters hottest
services is Shifting Abroad Delivery Containers for anyone relocating overseas.
Self Pack Worldwide transferring containers are a low cost alternate to high priced
Global removalists. Other works by using for shipping containers incorporate business
enterprise investing internationally seeking to transport bigger type shipments
cost efficiently from or to Melbourne. CargoMaster moves all sorts of Intercontinental
freight forwarders Melbourne by air freight and sea freight. Freight businesses or
freight carriers are the guys in control of the drivers and vehicles that move
freight all across the nation. There's a lots of variance in the businesses.
Some are significant national chains, such as R&L delivery costs, that provider
most areas of\nher explanationthe continental United states of america, let alone
Canada, Mexico, Puerto Rico, Hawaii, and in some cases Alaska.
"""

print(ask(template.format(article=ups), max_new_tokens=120))

In [ ]:
template2 = """What is the main topic of the article below?:

{article}
"""

print(ask(template2.format(article=ups), max_new_tokens=200))

In [ ]:
template2 = """Translate the article below for German:

{article}
"""

print(ask(template2.format(article=ups), max_new_tokens=900))